In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mglearn
%matplotlib inline
import seaborn as sns
import platform
from matplotlib import font_manager , rc

if platform.system() == 'Darwin':
  rc('font' , family = 'AppleGothic')
elif platform.system() == 'Windows':
  path = 'C:/Windows/Fonts/malgun.ttf'
  font_name = font_manager.FontProperties(fname = path).get_name()
  rc('font' , family = font_name)
else:
  print('모름')
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score , precision_score , recall_score , roc_auc_score , f1_score , confusion_matrix , roc_curve , precision_recall_curve

In [3]:
path = 'C:/k_digital/machine/source/house-prices-advanced-regression-techniques'
house = pd.read_csv(path + '/train.csv')
test = pd.read_csv(path + '/test.csv')
pred = pd.read_csv(path + '/sample_submission.csv')

In [4]:
house.drop('Id' , axis = 1 , inplace = True)

In [5]:
house1 = house[['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice']]

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
mm = MinMaxScaler()

In [7]:
house1.LotFrontage = house1.LotFrontage.fillna(70)
test.LotFrontage = test.LotFrontage.fillna(69)


In [8]:
test = test[['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold']]

In [9]:
house1.GarageYrBlt = house1.GarageYrBlt.fillna(house1.GarageYrBlt.mean())
test.GarageYrBlt = test.GarageYrBlt.fillna(test.GarageYrBlt.mean())

In [10]:
house1.MasVnrArea = house1.MasVnrArea.fillna(0)
test.MasVnrArea = test.MasVnrArea.fillna(0)

In [11]:
test = test.fillna(0)

In [12]:
data = house1.iloc[:,:-1]
target = house1.iloc[:,-1]

data.iloc[:,14] = -data.iloc[:,14]

In [13]:
mm.fit(data)

MinMaxScaler()

In [14]:
data_mm = mm.transform(data)

In [15]:
data_mm = pd.DataFrame(data = data_mm , columns = house1.columns[:-1])

In [26]:
from sklearn.ensemble import RandomForestRegressor

In [27]:
log_target = np.log(target)

# 랜덤 포레스트 모델 초기화
rf = RandomForestRegressor()

# 모델 학습 (로그 변환된 target 사용)
rf.fit(data_mm, log_target)

# 예측
predictions = rf.predict(data_mm)

In [28]:
predictions

array([12.24153041, 12.07344158, 12.29637668, ..., 12.43592376,
       11.82087865, 11.91684512])

In [29]:
pred.SalePrice = np.exp(predictions)

ValueError: Length of values (1460) does not match length of index (1459)

In [4]:
data = pd.read_csv('water_potability.csv')
df = data.copy()
df.dropna(subset = ['ph'] , inplace = True)
data.ph.fillna(df.ph.mean() , inplace = True)

def fit(model , data , target , fill_target):
    model.fit(data , target)
    return model.predict(fill_target)



from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression , Ridge

rid = Ridge()
rf = RandomForestRegressor()
lr = LinearRegression()


fill_Tri = data.dropna(subset = ['Trihalomethanes']).drop(['Trihalomethanes' , 'Potability' , 'Sulfate'] , axis = 1)
fill_Tri_target = data.dropna(subset = ['Trihalomethanes'])['Trihalomethanes']

fill_target = data[data.Trihalomethanes.isna()].dropna(axis = 1).drop('Potability' , axis = 1)


model = [rid , rf , lr]
result = []
for i in model:
    result.append(fit(i , fill_Tri , fill_Tri_target , fill_target))
    
Tri_fill = np.mean(result , axis = 0)

data.loc[data['Trihalomethanes'].isna(), 'Trihalomethanes'] = Tri_fill

fill_Sul = data.dropna(subset = ['Sulfate']).drop(['Potability' , 'Sulfate'] , axis = 1)
fill_Sul_target = data.dropna(subset = ['Sulfate'])['Sulfate']

fill_target_Sul = data[data.Sulfate.isna()].dropna(axis = 1).drop('Potability' , axis = 1)

Sulfate = []

for i in model:
    Sulfate.append(fit(i , fill_Sul , fill_Sul_target , fill_target_Sul))
    
Sul_fill = np.mean(Sulfate , axis = 0)

Sulfate = Sul_fill

data.loc[data['Sulfate'].isna(), 'Sulfate'] = Sulfate

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(data.iloc[:,:-1])
data2 = ss.transform(data.iloc[:,:-1])

from sklearn.model_selection import train_test_split
train_input , test_input , train_target , test_target = train_test_split(data2 , data.iloc[:,-1] , test_size = 0.2 , stratify = data.iloc[:,-1] , random_state = 0)


In [8]:
pd.DataFrame(data2 , columns = data.columns[:-1])

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,-6.043133e-16,0.259195,-0.139471,0.112415,0.952957,1.708954,-1.180651,1.305840,-1.286298
1,-2.289339e+00,-2.036414,-0.385987,-0.307694,0.331984,2.062575,0.270597,-0.638475,0.684218
2,6.928678e-01,0.847665,-0.240047,1.360594,-0.125485,-0.094032,0.781117,0.001411,-1.167365
3,8.409504e-01,0.547651,0.000493,0.592008,0.633890,-0.778830,1.255134,2.152428,0.848412
4,1.368569e+00,-0.464429,-0.460249,-0.363698,-0.648666,-0.343939,-0.824357,-2.181345,0.138786
...,...,...,...,...,...,...,...,...,...
3271,-1.641587e+00,-0.081758,2.916188,0.028027,0.717905,1.240155,-0.118075,0.018380,0.601124
3272,4.953704e-01,-0.085667,-0.534295,0.593290,0.305151,-0.417706,1.698560,0.074444,-1.497627
3273,1.591254e+00,-0.626829,1.270810,0.144017,-0.070759,0.072263,-0.981329,0.218615,-0.856007
3274,-1.329516e+00,1.041355,-1.144058,-0.517373,-0.053529,-0.288597,-0.942064,0.703257,0.950797


In [6]:
train_input , test_input , train_target , test_target = train_test_split(data2 , data.iloc[:,-1] , test_size = 0.2 , stratify = data.iloc[:,-1] , random_state = 0)

array([[-6.04313345e-16,  2.59194711e-01, -1.39470871e-01, ...,
        -1.18065057e+00,  1.30584045e+00, -1.28629758e+00],
       [-2.28933938e+00, -2.03641367e+00, -3.85986650e-01, ...,
         2.70597240e-01, -6.38475300e-01,  6.84217891e-01],
       [ 6.92867789e-01,  8.47664833e-01, -2.40047337e-01, ...,
         7.81116857e-01,  1.41086173e-03, -1.16736546e+00],
       ...,
       [ 1.59125368e+00, -6.26829230e-01,  1.27080989e+00, ...,
        -9.81329234e-01,  2.18614659e-01, -8.56006782e-01],
       [-1.32951593e+00,  1.04135450e+00, -1.14405809e+00, ...,
        -9.42063817e-01,  7.03256646e-01,  9.50797383e-01],
       [ 5.40150905e-01, -3.85462310e-02, -5.25811937e-01, ...,
         5.60940070e-01,  7.79999312e-01, -2.12445866e+00]])

In [5]:

from sklearn.svm import SVC

svc = SVC()

SVC.fit(train_input , train_target)

AttributeError: 'numpy.ndarray' object has no attribute '_validate_params'